In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

from tqdm import tqdm
from sklearn import datasets
import random, sys, os

In [3]:
import torch.optim as optim
from torch.utils import data
from torchvision import datasets, transforms

In [4]:
device = torch.device("cuda:0")
# device = torch.device("cuda:1")
# device = torch.device("cpu")

In [5]:
train_transform = transforms.Compose([
            transforms.ToTensor(),
        ])
test_transform = transforms.Compose([
            transforms.ToTensor(),
        ])

train_dataset = datasets.FashionMNIST(root="../../../../_Datasets/FMNIST/", train=True, download=True, transform=train_transform)
test_dataset = datasets.FashionMNIST(root="../../../../_Datasets/FMNIST/", train=False, download=True, transform=test_transform)

In [6]:
LR = 0.0001
BS = 200

In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BS, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BS, shuffle=False, num_workers=2)

In [8]:
## demo of train loader
xx, yy = iter(train_loader).next()
xx.shape

torch.Size([200, 1, 28, 28])

## Cuda -bmm2x2

In [9]:
import bmm2x2_cuda

In [10]:
class BMM2x2Function(torch.autograd.Function):
    @staticmethod
#     @torch.jit.ignore
    def forward(ctx, inputs, weights):
        outputs = bmm2x2_cuda.forward(inputs, weights)
        ctx.save_for_backward(inputs, weights)
        return outputs[0]
    
    @staticmethod
#     @torch.jit.ignore
    def backward(ctx, grad_output):
        inputs, weights = ctx.saved_tensors
        del_input, del_weights = bmm2x2_cuda.backward(
            inputs, 
            weights, 
            grad_output)
    
        return del_input, del_weights

In [11]:
class PairWeight2(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        assert input_dim%2 == 0, "Input dim must be even number"
        self.weight = torch.eye(2).unsqueeze(0).repeat_interleave(input_dim//2, dim=0)
        self.weight = nn.Parameter(self.weight)
        self.bmmfunc = BMM2x2Function()
        
    @torch.jit.ignore
    def bmm(self, x, w):
        return BMM2x2Function.apply(x, w)
        
    def forward(self, x):
        bs, dim = x.shape[0], x.shape[1]
        x = x.view(bs, -1, 2)
        x = self.bmm(x, self.weight)
        x = x.view(bs, -1)
        return x

## Cuda - Bilinear2x2

In [12]:
import bilinear2x2_cuda

In [13]:
del_input_list = []

In [14]:
class BiLinear2x2Function(torch.autograd.Function):
    @staticmethod
    def forward(ctx, inputs, weights):
        outputs = bilinear2x2_cuda.forward(inputs, weights)
        ctx.save_for_backward(inputs, weights)
        return outputs[0]

    @staticmethod
    def backward(ctx, grad_output):
        global del_input_list
        inputs, weights = ctx.saved_tensors
#         del_input, del_weights = bmm2x2_cuda.backward(
#             grad_output.contiguous(), 
#             grad_cell.contiguous(), 
#             grad_output.contiguous())
        del_input, del_weights = bilinear2x2_cuda.backward(
            inputs, 
            weights, 
            grad_output)
        del_input_list.append(del_input)
        return del_input, del_weights

In [15]:
class PairBilinear2(nn.Module):
    def __init__(self, dim, grid_width):
        super().__init__()
        self.dim = dim
        self.grid_width = grid_width
        
        self.num_pairs = self.dim // 2
        along_row = torch.linspace(0, 1, self.grid_width).reshape(1, -1).t()
        along_col = torch.linspace(0, 1, self.grid_width).reshape(-1, 1).t()
        
        self.Y = torch.stack([along_row+along_col*0, along_row*0+along_col])
        self.Y = torch.repeat_interleave(self.Y.unsqueeze(0), self.num_pairs, dim=0)
        self.Y = nn.Parameter(self.Y)
        
        self.pairW = torch.eye(2).unsqueeze(0).repeat_interleave(self.num_pairs, dim=0)
        self.pairW = nn.Parameter(self.pairW)
    
#     @torch.jit.ignore
#     def pairbl2x2(self, x, w):
#         return BiLinear2x2Function.apply(x, w)
    
#     @torch.jit.ignore
    def forward(self, x):
        bs = x.shape[0]
        
############# This block ########################
        ### this block is significantly faster
    
#         x = x.view(bs, -1, 2).transpose(0,1)
#         x = torch.bmm(x, self.pairW)
#         x = x.transpose(1,0)#.reshape(-1, 2)
        
############# OR This block ########################
        x = x.contiguous().view(bs, -1, 2)
#         x = BMM2x2Function.apply(x, self.pairW)
####################################################
#         x = x.view(bs, -1, 2)
        x = BiLinear2x2Function.apply(x, self.Y)
        x = x.view(bs, -1)
        return x

In [16]:
class FactorizedPairBilinearSpline_2(nn.Module):
    
    def __init__(self, input_dim, grid_width, num_layers=None):
        super().__init__()
        assert input_dim%2 == 0, "Input dim must be even number"
        self.input_dim = input_dim
        self.num_layers = int(np.ceil(np.log2(self.input_dim)))
        if num_layers is not None:
            self.num_layers = num_layers
            
        self.facto_nets = []
        for i in range(self.num_layers):
            net = PairBilinear2(self.input_dim, grid_width)
            self.facto_nets.append(net)
        self.facto_nets = nn.ModuleList(self.facto_nets)
            
    def forward(self, x):
        
        global del_input_list
        del_input_list = []
        
        ## swap first and then forward and reverse-swap
        bs = x.shape[0]
        y = x
#         for i in range(len(self.facto_nets)):
        for i, fn in enumerate(self.facto_nets):
            y = y.view(-1,2,2**(i)).permute(0, 2,1).contiguous().view(bs, -1)
            y = fn(y) 
            y = y.view(-1,2**(i),2).permute(0, 2,1).contiguous()
#         y = x + y ## this is residual addition... remove if only want feed forward
        return y.view(bs, -1)

In [17]:
# class PairBilinearBlock_2(FactorizedPairBilinearSpline_2):
    
#     def __init__(self, input_dim, grid_width):
#         num_layers = int(np.ceil(np.log2(input_dim)))
#         extra =  2**num_layers - input_dim
#         torch.manual_seed(123)
#         self.selector = torch.randperm(input_dim)[:extra]
        
#         super().__init__(2**num_layers, grid_width)
        
#     def forward(self, x):
#         '''
#         x should have dimension -> bs, M
#         '''
#         x = torch.cat((x, x[:, self.selector]), dim=1)
#         return super().forward(x)

## Fused 2x2 Ops

In [18]:
import fused2x2ops_cuda

In [19]:
recent_del_input = None

In [20]:
class FusedBiLinear2x2Function(torch.autograd.Function):
    @staticmethod
    def forward(ctx, inputs, weights, grids):
        outputs, input_buffer, dbg = fused2x2ops_cuda.bilinear2x2_forward(inputs, weights, grids)
#         print(dbg)

#         outputs, input_buffer = fused2x2ops_cuda.bilinear2x2_forward(inputs, weights, grids)
        ctx.save_for_backward(input_buffer, weights, grids)
        return outputs, dbg

    @staticmethod
    def backward(ctx, grad_output, a):
        global recent_del_input
        input_buffer, weights, grids = ctx.saved_tensors
#         del_input, del_weights = bmm2x2_cuda.backward(
#             grad_output.contiguous(), 
#             grad_cell.contiguous(), 
#             grad_output.contiguous())
        del_input, del_weights, del_grids, del_inputs = fused2x2ops_cuda.bilinear2x2_backward(
            input_buffer, 
            weights, 
            grids,
            grad_output)
    
        recent_del_input = del_inputs
        return del_input, del_weights, del_grids

In [83]:
class Fused2x2BiLinear(nn.Module):
    def __init__(self, dim, grid_width, num_layers = None):
        super().__init__()
        self.dim = dim
        self.grid_width = grid_width
        
        self.num_layers = int(np.ceil(np.log2(self.dim)))
        if num_layers is not None:
            self.num_layers = num_layers
        
        
        self.num_pairs = self.dim // 2
        
#         along_row = torch.linspace(0, 1, self.grid_width).reshape(1, -1)
#         along_col = torch.linspace(0, 1, self.grid_width).reshape(-1, 1)
        
        along_row = torch.linspace(0, 1, self.grid_width).reshape(1, -1).t()
        along_col = torch.linspace(0, 1, self.grid_width).reshape(-1, 1).t()
        
        self.Y = torch.stack([along_row+along_col*0, along_row*0+along_col])
        
        ### repeat same for num_pairs
        self.Y = torch.repeat_interleave(self.Y.unsqueeze(0), self.num_pairs, dim=0)
        ### repeat same for num_Layers
        self.Y = torch.repeat_interleave(self.Y.unsqueeze(0), self.num_layers, dim=0)
        
        print(self.Y.shape)
        self.Y = nn.Parameter(self.Y)
        
        ### repeat same for num_pairs
        self.pairW = torch.eye(2).unsqueeze(0).repeat_interleave(self.num_pairs, dim=0)
        ### repeat same for num_Layers
        self.pairW = self.pairW.unsqueeze(0).repeat_interleave(self.num_layers, dim=0)
        
        print(self.pairW.shape)
        self.pairW = nn.Parameter(self.pairW)
    
    def forward(self, x):
        self.Y.data.clamp_(-10, 10)
        x, self.dbg = FusedBiLinear2x2Function.apply(x.clone(), self.pairW, self.Y)
        return x

In [84]:
N = 16
fbl = Fused2x2BiLinear(N, 3, num_layers=1).to(device)

torch.Size([1, 8, 2, 3, 3])
torch.Size([1, 8, 2, 2])


In [85]:
# pbl = FactorizedPairBilinearSpline_2(N, 3, num_layers=1).to(device)

In [86]:
a = torch.randn(1, N).to(device)
y = fbl(a)

In [87]:
fbl.dbg

tensor([[[[ 0.,  0.,  1.],
          [ 1.,  2.,  3.],
          [ 2.,  4.,  5.],
          [ 3.,  6.,  7.],
          [ 4.,  8.,  9.],
          [ 5., 10., 11.],
          [ 6., 12., 13.],
          [ 7., 14., 15.]]]], device='cuda:0',
       grad_fn=<FusedBiLinear2x2FunctionBackward>)

In [88]:
y.shape

torch.Size([1, 16])

In [89]:
a

tensor([[ 0.6363, -0.2437, -2.7530,  1.5256,  0.8344, -0.3557,  0.6936, -0.1405,
          0.3859,  0.4446, -1.0249,  0.5198,  0.0668, -1.0162, -1.2000,  1.0106]],
       device='cuda:0')

In [90]:
y

tensor([[ 0.6363, -0.2437, -2.7530,  1.5256,  0.8344, -0.3557,  0.6936, -0.1405,
          0.3859,  0.4446, -1.0249,  0.5198,  0.0668, -1.0162, -1.2000,  1.0106]],
       device='cuda:0', grad_fn=<FusedBiLinear2x2FunctionBackward>)

In [91]:
y.mean().backward()

In [92]:
fbl.Y

Parameter containing:
tensor([[[[[0.0000, 0.0000, 0.0000],
           [0.5000, 0.5000, 0.5000],
           [1.0000, 1.0000, 1.0000]],

          [[0.0000, 0.5000, 1.0000],
           [0.0000, 0.5000, 1.0000],
           [0.0000, 0.5000, 1.0000]]],


         [[[0.0000, 0.0000, 0.0000],
           [0.5000, 0.5000, 0.5000],
           [1.0000, 1.0000, 1.0000]],

          [[0.0000, 0.5000, 1.0000],
           [0.0000, 0.5000, 1.0000],
           [0.0000, 0.5000, 1.0000]]],


         [[[0.0000, 0.0000, 0.0000],
           [0.5000, 0.5000, 0.5000],
           [1.0000, 1.0000, 1.0000]],

          [[0.0000, 0.5000, 1.0000],
           [0.0000, 0.5000, 1.0000],
           [0.0000, 0.5000, 1.0000]]],


         [[[0.0000, 0.0000, 0.0000],
           [0.5000, 0.5000, 0.5000],
           [1.0000, 1.0000, 1.0000]],

          [[0.0000, 0.5000, 1.0000],
           [0.0000, 0.5000, 1.0000],
           [0.0000, 0.5000, 1.0000]]],


         [[[0.0000, 0.0000, 0.0000],
           [0.5000, 0.5000, 0

In [93]:
pbl = FactorizedPairBilinearSpline_2(N, 3, num_layers=1).to(device)

In [94]:
pbl.facto_nets[0].Y

Parameter containing:
tensor([[[[0.0000, 0.0000, 0.0000],
          [0.5000, 0.5000, 0.5000],
          [1.0000, 1.0000, 1.0000]],

         [[0.0000, 0.5000, 1.0000],
          [0.0000, 0.5000, 1.0000],
          [0.0000, 0.5000, 1.0000]]],


        [[[0.0000, 0.0000, 0.0000],
          [0.5000, 0.5000, 0.5000],
          [1.0000, 1.0000, 1.0000]],

         [[0.0000, 0.5000, 1.0000],
          [0.0000, 0.5000, 1.0000],
          [0.0000, 0.5000, 1.0000]]],


        [[[0.0000, 0.0000, 0.0000],
          [0.5000, 0.5000, 0.5000],
          [1.0000, 1.0000, 1.0000]],

         [[0.0000, 0.5000, 1.0000],
          [0.0000, 0.5000, 1.0000],
          [0.0000, 0.5000, 1.0000]]],


        [[[0.0000, 0.0000, 0.0000],
          [0.5000, 0.5000, 0.5000],
          [1.0000, 1.0000, 1.0000]],

         [[0.0000, 0.5000, 1.0000],
          [0.0000, 0.5000, 1.0000],
          [0.0000, 0.5000, 1.0000]]],


        [[[0.0000, 0.0000, 0.0000],
          [0.5000, 0.5000, 0.5000],
          [1.0000,

In [95]:
y1 = pbl(a)

In [96]:
y1

tensor([[ 0.6363, -0.2437, -2.7530,  1.5256,  0.8344, -0.3557,  0.6936, -0.1405,
          0.3859,  0.4446, -1.0249,  0.5198,  0.0668, -1.0162, -1.2000,  1.0106]],
       device='cuda:0', grad_fn=<ViewBackward>)

In [97]:
a

tensor([[ 0.6363, -0.2437, -2.7530,  1.5256,  0.8344, -0.3557,  0.6936, -0.1405,
          0.3859,  0.4446, -1.0249,  0.5198,  0.0668, -1.0162, -1.2000,  1.0106]],
       device='cuda:0')

In [98]:
y1.mean().backward()

In [99]:
fbl.Y.grad[0][1]

tensor([[[ 0.0000, -0.4274,  0.8341],
         [ 0.0000,  0.3617, -0.7059],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.0000, -0.4274,  0.8341],
         [ 0.0000,  0.3617, -0.7059],
         [ 0.0000,  0.0000,  0.0000]]], device='cuda:0')

In [100]:
pbl.facto_nets[0].Y.grad[1]

tensor([[[ 0.0000, -0.4274,  0.8341],
         [ 0.0000,  0.3617, -0.7059],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.0000, -0.4274,  0.8341],
         [ 0.0000,  0.3617, -0.7059],
         [ 0.0000,  0.0000,  0.0000]]], device='cuda:0')

In [101]:
# fbl.Y.grad[0] - pbl.Y.grad.transpose(-1, -2)
# fbl.Y.grad[0] - pbl.Y.grad

In [102]:
pbl.facto_nets[0].pairW.grad[1]

TypeError: 'NoneType' object is not subscriptable

In [103]:
fbl.pairW.grad[0][1]

tensor([[0., 0.],
        [0., 0.]], device='cuda:0')

In [104]:
pbl.facto_nets[0].pairW[1]

tensor([[1., 0.],
        [0., 1.]], device='cuda:0', grad_fn=<SelectBackward>)

In [105]:
fbl.pairW[0][1]

tensor([[1., 0.],
        [0., 1.]], device='cuda:0', grad_fn=<SelectBackward>)

## Optimizing both models

In [106]:
N = 16
L = 2
a = torch.randn(5, N).to(device)
t = torch.randn_like(a)

In [107]:
def criterion(y, t):
    return ((y-t)**2).sum()

In [108]:
fbl = Fused2x2BiLinear(N, 2, num_layers=L).to(device)

torch.Size([2, 8, 2, 2, 2])
torch.Size([2, 8, 2, 2])


In [109]:
fbl_opt = torch.optim.Adam(fbl.parameters(), lr=0.01)

In [110]:
pbl = FactorizedPairBilinearSpline_2(N, 2, num_layers=L).to(device)

In [111]:
pbl_opt = torch.optim.Adam(pbl.parameters(), lr=0.01)

In [112]:
# fbl.pairW.data = torch.randn_likee(fbl.pairW.data)

In [113]:
# pbl.pairW.data = fbl.pairW.data[0].clone()

In [114]:
fbl.pairW.data

tensor([[[[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]]],


        [[[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]],

         [[1., 0.],
          [0., 1.]]]], device='cuda:0')

In [115]:
pbl.facto_nets[0].pairW.data

tensor([[[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]]], device='cuda:0')

In [116]:
fbl.Y.data = torch.randn_like(fbl.Y.data)

for i in range(L):
    pbl.facto_nets[i].Y.data = fbl.Y.data[i].clone()#.transpose(-1,-2).contiguous()

In [117]:
fbl.Y.data[0][0]

tensor([[[-1.2788,  0.6000],
         [ 0.2409, -2.0209]],

        [[-0.8392,  0.3698],
         [ 0.3418,  0.2273]]], device='cuda:0')

In [118]:
pbl.facto_nets[0].Y.data[0]

tensor([[[-1.2788,  0.6000],
         [ 0.2409, -2.0209]],

        [[-0.8392,  0.3698],
         [ 0.3418,  0.2273]]], device='cuda:0')

### Repeat from here

In [119]:
y = fbl(a)

In [120]:
fbl_opt.zero_grad()
criterion(y,t).backward()

In [121]:
y1 = pbl(a)

In [122]:
pbl_opt.zero_grad()
criterion(y1,t).backward()

In [123]:
y1, y

(tensor([[-1.4147e+00,  1.9673e-01,  3.0928e+00,  8.6186e-02,  8.4433e+00,
           2.4715e+00, -8.9128e+00,  7.5077e-01, -9.5820e-01,  1.7758e+00,
           1.6081e+00, -2.7869e+00,  5.7434e+00, -2.2289e+01,  1.4339e+00,
           7.0539e-01],
         [ 1.1000e+00, -8.2764e-01,  1.5004e-01, -5.3716e-01,  1.5792e+01,
           2.3115e+00, -1.5834e+01,  1.6151e+00, -2.5385e+00,  1.3427e+00,
           3.6838e+00, -1.8933e+00, -6.1707e+00,  8.2751e+00, -4.6740e+00,
          -2.1153e+00],
         [-5.9749e+00, -2.1276e+00,  1.2178e+01,  1.4900e+00, -3.8751e+01,
          -1.1987e+02,  4.3930e+01, -3.5777e+01,  2.8179e+00,  8.4018e-01,
          -5.7506e+00,  4.2089e+00,  5.2919e+00, -2.0481e+01,  1.3606e+00,
           1.7360e+00],
         [ 1.0949e+01,  4.8650e-01, -9.5503e+00,  6.9283e-01,  1.4128e+00,
          -1.4894e+01,  1.2217e+00, -3.3773e+00,  4.0301e-01,  7.5601e-01,
          -1.4826e+00,  2.8505e+00,  1.1386e+01, -4.1574e+01,  5.4404e+00,
           4.0386e+00],
    

In [124]:
# a

In [125]:
'''
OBSERVATIONS:
1. The kernel for copy operation works.; Now try matrix multiplication.. also works
2. 
'''

'\nOBSERVATIONS:\n1. The kernel for copy operation works.; Now try matrix multiplication.. also works\n2. \n'

In [126]:
y1.data-y.data

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       device='cuda:0')

In [127]:
# fbl.Y.grad

In [128]:
# pbl.Y.grad

In [129]:
fbl_opt.step()
pbl_opt.step()

In [130]:
# fbl.Y.grad[0] - pbl.Y.grad.transpose(-1, -2)
# torch.abs(fbl.Y.grad[0] - pbl.Y.grad).abs().sum()

In [131]:
torch.abs(fbl.Y.data[0] - pbl.facto_nets[0].Y.data).abs().sum()

tensor(0., device='cuda:0')

In [132]:
torch.abs(fbl.Y.grad[0] - pbl.facto_nets[0].Y.grad).abs().sum()

tensor(0.0007, device='cuda:0')

In [133]:
torch.abs(fbl.pairW.data[0] - pbl.facto_nets[0].pairW.data).abs().sum()

tensor(0., device='cuda:0')

In [134]:
torch.abs(fbl.pairW.grad[0] - pbl.facto_nets[0].pairW.grad).abs().sum()

TypeError: unsupported operand type(s) for -: 'Tensor' and 'NoneType'

In [135]:
# pbl.pairW.grad[1]

In [136]:
# fbl.pairW.grad[0][1]

In [137]:
pbl.facto_nets[0].pairW

Parameter containing:
tensor([[[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]]], device='cuda:0', requires_grad=True)

In [138]:
fbl.pairW[0]

tensor([[[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]]], device='cuda:0', grad_fn=<SelectBackward>)

In [139]:
torch.abs(fbl.pairW.data[0] - pbl.facto_nets[0].pairW.data)

tensor([[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]], device='cuda:0')

## dissecting the difference in first gradient

In [140]:
fbl.Y.grad[0] - pbl.facto_nets[0].Y.grad

tensor([[[[ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]]],


        [[[ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]]],


        [[[ 0.0000e+00, -2.4414e-04],
          [ 0.0000e+00,  1.2207e-04]],

         [[ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]]],


        [[[ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]]],


        [[[ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]]],


        [[[ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]]],


        [[[ 0.0000e+00, -3.0518e-05],
          [ 0.

In [141]:
fbl.Y.grad[0][0], pbl.facto_nets[0].Y.grad[0]

(tensor([[[-2831.5654,  1168.5298],
          [ 1595.1970,  -766.5468]],
 
         [[ -229.0080,    95.9249],
          [  100.6610,   -41.4758]]], device='cuda:0'),
 tensor([[[-2831.5654,  1168.5298],
          [ 1595.1970,  -766.5468]],
 
         [[ -229.0080,    95.9249],
          [  100.6610,   -41.4758]]], device='cuda:0'))

In [142]:
fbl.Y.grad[1] - pbl.facto_nets[1].Y.grad

tensor([[[[0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.]]],


        [[[0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.]]],


        [[[0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.]]],


        [[[0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.]]],


        [[[0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.]]],


        [[[0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.]]],


        [[[0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.]]],


        [[[0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.]]]], device='cuda:0')

In [143]:
recent_del_input[1]#.sum()

tensor([[-1.3237e+01,  4.6712e-01, -1.1120e+01,  6.1231e-01, -8.3015e+01,
          1.7139e+01, -1.4816e+02, -1.4131e+01,  2.8407e-01, -4.5071e+00,
         -3.1886e+00,  7.8135e+00, -2.2336e+01, -3.0173e+02, -2.2165e+01,
         -1.8859e+02],
        [ 6.4398e+00,  1.4490e+00,  2.4738e+00, -1.3046e+00, -1.0983e+02,
          6.4152e+00, -9.0764e+02, -8.4267e+01,  2.6650e+01, -7.3618e-01,
         -4.2920e+01,  2.8359e-01,  3.1488e+01,  3.1981e+01,  3.2400e+01,
         -6.8669e+01],
        [-1.9749e+02, -1.5759e+01, -8.7394e+01,  6.7024e+00,  3.5928e+02,
         -2.4616e+03, -4.6154e+03, -7.1210e+03, -3.9389e+01,  1.0613e+01,
         -4.8011e+01, -1.6281e+01, -2.1880e+01, -1.5191e+02, -2.0934e+01,
         -3.0504e+02],
        [ 8.7606e+01, -2.0669e+00, -1.8628e+02, -1.3838e+00, -1.3782e+00,
          1.2751e+02, -2.2942e+01,  2.6093e+02,  6.1948e-01,  5.2663e+00,
          2.8668e-01, -8.2319e+00, -4.4537e+01, -3.3690e+02, -4.5247e+01,
         -1.0804e+03],
        [-7.1770e+02

In [144]:
del_input_list[0]#.sum()

tensor([[[-1.3237e+01, -1.1120e+01],
         [ 4.6712e-01,  6.1231e-01],
         [-8.3015e+01, -1.4816e+02],
         [ 1.7139e+01, -1.4131e+01],
         [ 2.8407e-01, -3.1886e+00],
         [-4.5071e+00,  7.8135e+00],
         [-2.2336e+01, -2.2165e+01],
         [-3.0173e+02, -1.8859e+02]],

        [[ 6.4398e+00,  2.4738e+00],
         [ 1.4490e+00, -1.3046e+00],
         [-1.0983e+02, -9.0764e+02],
         [ 6.4152e+00, -8.4267e+01],
         [ 2.6650e+01, -4.2920e+01],
         [-7.3618e-01,  2.8359e-01],
         [ 3.1488e+01,  3.2400e+01],
         [ 3.1981e+01, -6.8669e+01]],

        [[-1.9749e+02, -8.7394e+01],
         [-1.5759e+01,  6.7024e+00],
         [ 3.5928e+02, -4.6154e+03],
         [-2.4616e+03, -7.1210e+03],
         [-3.9389e+01, -4.8011e+01],
         [ 1.0613e+01, -1.6281e+01],
         [-2.1880e+01, -2.0934e+01],
         [-1.5191e+02, -3.0504e+02]],

        [[ 8.7606e+01, -1.8628e+02],
         [-2.0669e+00, -1.3838e+00],
         [-1.3782e+00, -2.2942e+

In [145]:
del_input_list[0].reshape(5, -1, 2, 2).transpose(-1, -2).reshape(5, -1) - recent_del_input[1]

tensor([[-9.5367e-07,  2.9802e-08,  0.0000e+00,  5.9605e-08,  0.0000e+00,
          0.0000e+00,  1.5259e-05,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  6.1035e-05,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          3.8147e-06,  0.0000e+00, -1.9073e-06,  1.9073e-06, -3.8147e-06,
          0.0000e+00],
        [ 1.5259e-05,  0.0000e+00, -7.6294e-06,  0.0000e+00, -3.0518e-05,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  1.9073e-06,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  1.5259e-05,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  5.9605e-08,  0.0000e+00,
         -1.1921e-07,  0.0000e+00,  0.0000e+00,  3.0518e-05,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00

## Model

In [174]:
class FusedPairBilinearBlock(nn.Module):
    
    def __init__(self, input_dim, grid_width):
        super().__init__()
        self.input_dim = input_dim
        self.num_layers = int(np.ceil(np.log2(input_dim)))
        
        extra =  2**self.num_layers - input_dim
        torch.manual_seed(123)
        self.selector = torch.randperm(self.input_dim)[:extra]
        
        self.fused_pair_bilinear = Fused2x2BiLinear(2**self.num_layers, grid_width=grid_width, )#num_layers=5)
        
        
    def forward(self, x):
        '''
        x should have dimension -> bs, M
        '''
        x = torch.cat((x, x[:, self.selector]), dim=1)
        return self.fused_pair_bilinear(x)

In [175]:
class BiasLayer(nn.Module):
    def __init__(self, dim, init_val=0):
        super().__init__()
        self.bias = nn.Parameter(torch.ones(dim)*init_val)
        
    def forward(self, x):
        return x+self.bias

In [183]:
class FactorNet2(nn.Module):
    def __init__(self):
        super().__init__()
        H = 512
        self.bias = nn.Linear(784, H)
        self.bn1 = nn.BatchNorm1d(H)
        self.fc = nn.Linear(H, 10)
        self.la1 = FusedPairBilinearBlock(H, grid_width=3)
#         self.la1 = PairBilinearBlock_2(H, grid_width=3)

#         self.bias = BiasLayer(784)
#         self.la1 = FusedPairBilinearBlock(784, grid_width=3)
# #         self.la1 = PairBilinearBlock(784, grid_width=3)
#         self.bn1 = nn.BatchNorm1d(1024)
#         self.fc = nn.Linear(1024, 10)
        
    def forward(self, x):
        x = self.bias(x)
        x = self.la1(x)
#         x = self.bn1(x)
        x = torch.relu(x)
        x = self.fc(x)
        return x

In [184]:
torch.manual_seed(0)
model = FactorNet2().to(device)

model

torch.Size([9, 256, 2, 3, 3])
torch.Size([9, 256, 2, 2])


FactorNet2(
  (bias): Linear(in_features=784, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=512, out_features=10, bias=True)
  (la1): FusedPairBilinearBlock(
    (fused_pair_bilinear): Fused2x2BiLinear()
  )
)

In [185]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00003)

In [186]:
losses = []
train_accs = []
test_accs = []
EPOCHS = 20

for epoch in range(EPOCHS):
    
    train_acc = 0
    train_count = 0
    i = -1
    for xx, yy in tqdm(train_loader):
        i += 1 
        xx = xx.view(xx.shape[0], -1)
        xx, yy = xx.to(device), yy.to(device)

        yout = model(xx)
        
        if torch.any(torch.isnan(yout.data)):
            print(f"{i},Yout; NAN", flush=True)
            break
            
        loss = criterion(yout, yy)
        
        if torch.any(torch.isnan(loss.data)):
            print(f"{i},loss; NAN", flush=True)
            break
        
        optimizer.zero_grad()
        loss.backward()

        if torch.any(torch.isnan(model.la1.fused_pair_bilinear.Y.grad)):
            print(f"{i}, NAN Grid back grad; loss {float(loss)}", flush=True)
            break
        if torch.any(torch.isnan(model.la1.fused_pair_bilinear.pairW.grad)):
            print(f"{i}, NAN Weight back grad; loss {float(loss)}", flush=True)
            break

        
        optimizer.step()

        losses.append(float(loss))

        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        train_acc += correct
        train_count += len(outputs)
        
        if torch.any(torch.isnan(model.la1.fused_pair_bilinear.Y)):
            print(f"{i}, NAN Grid update; loss {float(loss)}", flush=True)
            break
        if torch.any(torch.isnan(model.la1.fused_pair_bilinear.pairW)):
            print(f"{i}, NAN Weight update; loss {float(loss)}", flush=True)
            break

    train_accs.append(float(train_acc)/train_count*100)
    train_acc = 0
    train_count = 0

    print(f'Epoch: {epoch},  Loss:{float(loss)}')
    test_count = 0
    test_acc = 0
    for xx, yy in tqdm(test_loader):
        xx = xx.view(xx.shape[0], -1)
        xx, yy = xx.to(device), yy.to(device)
        with torch.no_grad():
            yout = model(xx)
        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        test_acc += correct
        test_count += len(xx)
    test_accs.append(float(test_acc)/test_count*100)
    print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
    print()

### after each class index is finished training
print(f'\t-> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}')

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 0,  Loss:3.2544753551483154


  0%|          | 0/300 [00:00<?, ?it/s]

Train Acc:63.77%, Test Acc:70.41%



  3%|▎         | 9/300 [00:00<00:10, 27.36it/s]

...

Epoch: 1,  Loss:2.1953336815572767e+23


  0%|          | 0/300 [00:00<?, ?it/s]

Train Acc:69.91%, Test Acc:64.15%

0, NAN Grid back grad; loss 1.398152815470647e+27


  0%|          | 0/300 [00:00<?, ?it/s]


ZeroDivisionError: float division by zero

In [187]:
model.la1.fused_pair_bilinear.Y.data.max()

tensor(1.0163, device='cuda:0')

In [191]:
torch.any(torch.isnan(model.la1.fused_pair_bilinear.Y.data))

tensor(False, device='cuda:0')

In [192]:
yout = model(xx)

In [195]:
torch.any(torch.isnan(yout))

tensor(False, device='cuda:0')

In [196]:
yy

tensor([7, 1, 4, 5, 8, 3, 5, 7, 6, 4, 5, 9, 0, 0, 6, 9, 4, 1, 7, 0, 4, 1, 6, 8,
        6, 6, 5, 0, 2, 1, 8, 1, 9, 6, 5, 6, 6, 4, 0, 8, 6, 3, 4, 2, 4, 0, 7, 4,
        8, 9, 7, 9, 1, 1, 5, 7, 7, 3, 7, 1, 8, 4, 0, 2, 7, 3, 5, 4, 4, 9, 9, 8,
        4, 1, 9, 0, 7, 9, 0, 3, 6, 0, 2, 5, 0, 9, 5, 9, 0, 9, 1, 3, 4, 5, 4, 9,
        4, 3, 0, 5, 8, 2, 6, 2, 5, 4, 5, 7, 7, 4, 1, 8, 0, 8, 8, 9, 3, 1, 2, 7,
        7, 3, 1, 4, 4, 0, 6, 3, 8, 4, 5, 4, 0, 3, 2, 0, 5, 3, 4, 0, 3, 5, 5, 8,
        9, 1, 3, 0, 1, 2, 2, 2, 6, 7, 3, 0, 7, 6, 8, 7, 1, 2, 1, 2, 4, 4, 5, 9,
        0, 0, 8, 8, 9, 3, 6, 5, 3, 4, 8, 3, 9, 8, 0, 9, 3, 4, 5, 5, 1, 5, 9, 7,
        7, 7, 6, 8, 3, 5, 6, 6], device='cuda:0')

In [197]:
loss = criterion(yout, yy)

In [205]:
yout.argmax()//10

tensor(134, device='cuda:0')

In [206]:
yout[134]

tensor([ 1.1605e+29, -7.1041e+28, -1.0751e+29,  9.6640e+27, -1.3566e+29,
         1.1653e+29,  1.0879e+29,  1.7212e+29, -1.9855e+28,  1.7179e+29],
       device='cuda:0', grad_fn=<SelectBackward>)

In [218]:
yout[134]

tensor([ 1.1605e+29, -7.1041e+28, -1.0751e+29,  9.6640e+27, -1.3566e+29,
         1.1653e+29,  1.0879e+29,  1.7212e+29, -1.9855e+28,  1.7179e+29],
       device='cuda:0', grad_fn=<SelectBackward>)

In [219]:
model.la1.fused_pair_bilinear.Y.data

tensor([[[[[ 3.8739e-03, -7.1850e-03, -9.2713e-03],
           [ 5.0880e-01,  4.9659e-01,  5.0152e-01],
           [ 1.0121e+00,  1.0031e+00,  1.0011e+00]],

          [[-2.2405e-03,  5.0718e-01,  1.0115e+00],
           [-5.2883e-03,  4.9287e-01,  9.8861e-01],
           [-8.6327e-03,  4.9361e-01,  9.9927e-01]]],


         [[[ 2.1228e-03, -7.1292e-04, -8.8196e-03],
           [ 5.0610e-01,  5.0579e-01,  4.9202e-01],
           [ 1.0066e+00,  1.0050e+00,  9.9415e-01]],

          [[-1.5808e-03,  5.0555e-01,  1.0099e+00],
           [-4.6353e-03,  5.0876e-01,  1.0075e+00],
           [-4.6236e-03,  5.0164e-01,  1.0042e+00]]],


         [[[-4.0497e-03,  3.1962e-03, -1.1414e-02],
           [ 5.0873e-01,  4.9291e-01,  5.1071e-01],
           [ 1.0103e+00,  9.9000e-01,  1.0000e+00]],

          [[-1.8174e-03,  5.0718e-01,  1.0113e+00],
           [-1.7545e-03,  4.9468e-01,  9.8872e-01],
           [-9.1505e-03,  5.0697e-01,  1.0000e+00]]],


         ...,


         [[[ 5.7747e-03,  1.67